# What to check

* Tree crew = PO notification, grizzley flats, felling crew
* Arc and bio not blank
* WLPZ and inlieu cant not be blank

* RPF boundary. Multiple steps
* 1) If RPF is no then we are good
* 2) If yes then then we need a RPF date
* 3) If PLS is a no we are good
* 4) If if then we need a date

* H/O contact date

In [1]:
import pandas as pd
import os
import glob
import numpy as np

# Variables

In [29]:
excel = '.xlsx'
csv = '.csv'

# central and soouth central imports

In [3]:
central_path = r"C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker*"
load_c = glob.glob(central_path + excel)
central = pd.read_excel(max(load_c, key=os.path.getctime), parse_dates=True)

south_path = r"C:\Users\jacque.trahan\Downloads\South Central Division 2021 Debris Removal Tracker*"
load_s = glob.glob(south_path + excel)
south = pd.read_excel(max(load_s, key=os.path.getctime), parse_dates=True)

df = pd.concat([central, south])


df['RPF Boundary Review Date'] = pd.to_datetime(df['RPF Boundary Review Date'].fillna(pd.NaT), errors='coerce')
df['PLS Boundary Review Date'] = pd.to_datetime(df['PLS Boundary Review Date'].fillna(pd.NaT), errors='coerce')

# df['RPF Boundary Review Date'] = pd.to_datetime(df['RPF Boundary Review Date']).dt.date
# df['PLS Boundary Review Date'] = pd.to_datetime(df['PLS Boundary Review Date']).dt.date

In [28]:
# df.info(verbose=True)

# RPF function check

In [23]:
def RPF(df):
    '''Check for RPF info'''
    if pd.isna(df['RPF Boundary Review Needed']):
        return "RPF status is blank"
    elif df['RPF Boundary Review Needed'] =='No':
        return "RPLF staus is No"
    elif df['RPF Boundary Review Needed'] =='Yes' and pd.isna(df['RPF Boundary Review Date']):
        return "RPF needs a date"
    elif df['RPF Boundary Review Needed'] =='Yes' and df['PLS Boundary Review Needed'] =='Yes' and (pd.isna(df['PLS Boundary Review Date'])):
        return 'PLS needs date'
    elif  df['RPF Boundary Review Needed'] =='Yes' and pd.isna(df['PLS Boundary Review Needed']):
        return 'PLS needs review'
    else:
        return "Everything checked for PLS and RPF"
    
    
def wlpz_inLieu(df):
    """WLPZ and In Liue checks"""
    if pd.isna(df['WLPZ Zone?']) & pd.isna(df['In lieu practices?']):
        return "Missing WLPZ AND In lieu"
    elif pd.isna(df['WLPZ Zone?'])
        return "Missing WLPZ"
    elif  pd.isna(df['In lieu practices?']):
        return ' Missing In lieu'
    else: 
        return 'WLPZ and In Liue completed'

In [26]:
Tree_runway = df.copy()
Tree_runway = Tree_runway[['APN_ROW Segment','Street #', 'Street Name', 'Structural Status','County', 'Tree Crew','Debris Crew',
                          'Arch Assessment', 'Bio Assessment',
                          'RPF Boundary Review Needed', 'RPF Boundary Review Date','RPF Hold',
                          'PLS Boundary Review Needed', 'PLS Boundary Review Date',
                          'WLPZ Zone?', 'In lieu practices?', 'Salmon Habitat', 'Contact Notes']]

Tree_runway.insert(Tree_runway.columns.get_loc('Salmon Habitat')+1, 'What is needed for RPF/PLS',
                  Tree_runway.apply(RPF,axis=1))

Tree_runway.insert(Tree_runway.columns.get_loc('What is needed for RPF/PLS')+1, 'What is needed for WLPZ/In Lieu',
                  Tree_runway.apply(wlpz_inLieu,axis=1))

In [27]:
Tree_runway.to_excel('Check Tree Runway crews.xlsx')